In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import pickle
import sys
import datetime as dt
import ca_util as ca
from sklearn import preprocessing

In [2]:
# 파라미터값을 설정합니다.(conf 파일 적용 예정)
output_dir = './output/'
data_dir = './data/'
seed = 0

parameter_id = 'default'
datafile = 'dataset_all.csv'
resp_success = ['지속(정책)', '지속사용(고객혜택제공)', '지속(혜택)']

# 서비스 위해 학습 환경 저장 합니다.
preprocess_object = {}       # 학습 환경 전체를 저장합니다.
preprocess_del_col = []      # 학습 시 삭제한 컬럼 목록 저장합니다.




In [3]:
dataset = pd.read_csv("dataset/cam_dataset_all.csv", delimiter=",", encoding='UTF8')

In [4]:
dataset

,고객번호,인터넷결합여부,IPTV결합여무,SOP결합여부,PSTN결합여부,와이브로결합여부,모바일결합여부,PSTN회선수,모바일회선수,SOIP회선수,...,최근3개월데이터충전횟수,최근3개월패박사용횟수,스폰서할인유형,판매유형,사용유형,납무방법,반응1,반응2,반응3,반응4
0,87HL036LE93,Y,Y,NaN,Y,NaN,Y,1,1,2,...,0,0,_,_,일반,은행계좌 자동이체,SR불필요,SR미발행,혜택제공,지속사용(고객혜택제공)
1,8MYL79EHX9Z,Y,Y,NaN,Y,NaN,Y,2,0,0,...,0,0,_,_,일반,신용카드,SR불필요,SR미발행,혜택제공,지속사용(고객혜택제공)
2,7QLKZUQHXHJ,Y,Y,NaN,N,NaN,Y,0,1,0,...,1,0,_,_,일반,신용카드,SR불필요,SR미발행,혜택제공,지속사용(고객혜택제공)
3,BE1L6Y4H2NG,N,N,NaN,N,NaN,N,0,0,0,...,0,0,_,_,일반,은행계좌 자동이체,고객측사유,고객측사유_기타,KT,해지
4,9KWL716JWKU,N,N,NaN,N,NaN,N,1,0,0,...,0,0,_,_,주택용,은행계좌 자동이체,SR불필요,SR미발행,혜택제공,지속사용(고객혜택제공)
5,9KWL716JWKU,N,N,NaN,N,NaN,N,1,0,0,...,0,0,_,_,주택용,은행계좌 자동이체,SR불필요,SR미발행,혜택제공,지속사용(고객혜택제공)
6,9YNKZH0K3HS,N,N,NaN,N,NaN,N,1,1,0,...,0,0,_,_,주택용,신용카드,회선품질불만,품질_기타,미사용,해지
7,AFHL004LCNQ,N,N,NaN,N,NaN,N,1,0,0,...,0,0,_,_,주택용,신용카드,불필요,사용저조/환경변화,미사용,해지
8,AEEL76LIMR4,N,N,NaN,N,NaN,N,1,0,0,...,0,0,_,_,업무용,은행계좌 자동이체,고객측사유,가족/지인등권유,KT,해지
9,DSUL7HCHY75,Y,Y,NaN,N,NaN,Y,0,0,1,...,0,0,_,_,주택용,신용카드,고객측사유,고객기기문제,미사용,해지


In [5]:

def resp_func (resp):
    if resp in resp_success:
        return 'Y'
    else:
        return 'N'
    

In [6]:
dataset['resp'] = dataset['반응4'].map(resp_func)



In [7]:
for col in dataset.columns:
    # '_ID' 항목 삭제 합니다.
    if col.upper().endswith('ID') or col.upper().endswith('_ID.1'):
        del dataset[col]
        preprocess_del_col.append(col)
    # '_DATE' 항목 삭제 합니다.
    elif col.upper().endswith('일자') or col.upper().endswith('_DATE') or col.upper().endswith('_DATE.1') or col.upper().endswith('_DT.1'):
        del dataset[col]
        preprocess_del_col.append(col)
    # 범주가 1 이하 항목 삭제 합니다.
    elif dataset[col].value_counts().shape[0] <=1:
        del dataset[col]
        preprocess_del_col.append(col)
    # 범주범위가 특정 % 이상 항목 삭제 합니다.
    elif dataset[col].value_counts().shape[0]/dataset.shape[0] > 0.50:
        del dataset[col]
        preprocess_del_col.append(col)
    # Others 항목 처리 합니다.
    else:
    	# 데이터 중 99999 -> 0 변환 합니다.
    	dataset[col] = dataset[col].replace(99999,0)
    	# 데이터 중 999 -> 0 변환 합니다.
    	dataset[col] = dataset[col].replace(999,0)

In [8]:
# 추가 삭제컬럼 입니다.
del_list = ['고객번호','반응1','반응2','반응3','반응4']
for del_col in del_list:
    try:
        del dataset[del_col]
        preprocess_del_col.append(del_col)
    except Exception as e:
        print('Deleting Exception :',e)

# 예외 처리 합니다.
#dataset['AGE_ITG_CD'] = dataset['AGE_ITG_CD'].astype(str)

# 삭제 항목 학습 환경에 저장합니다.
preprocess_object['del_col'] = preprocess_del_col

# 데이터중 null -> 0 처리 합니다.
dataset = dataset.fillna(0)

# 훈련/테스트 데이터 분리 후 전체/훈련/테스트 데이터 저장 합니다.
train_dataset = dataset.sample(frac=0.8, random_state=seed)
test_dataset = dataset.drop(train_dataset.index)

In [9]:
# 데이터 저장합니다.
dataset.to_csv('./output/preprocess_dataset_all.csv', mode='w', encoding='utf-8', header=1, index=0)
train_dataset.to_csv('./output/train_dataset.csv', mode='w', encoding='utf-8', header=1, index=0)
test_dataset.to_csv('./output/test_dataset.csv', mode='w', encoding='utf-8', header=1, index=0)

# 연속형/이산형 데이터 분리 합니다.
number_df = dataset.select_dtypes(include=['int64','float64'])
object_df = dataset.select_dtypes(include=['object'])

# 연속형 데이터 표준화 합니다.
scaler = preprocessing.StandardScaler()
scaler.fit(number_df)

# scaler 학습 환경에 저장 합니다.
preprocess_object['scaler'] = scaler

# 이산형 데이터 OneHot Encoding 합니다.
ca.log_print('OneHot Encoding 처리 합니다.')
categorical = list(object_df.columns)
for cat in categorical:
    oh_encoder = {}

    # LabelEncoder 사용 합니다.
    le = preprocessing.LabelEncoder()       # LabelEncoder 사용을 위해 선언합니다.
    le.fit(dataset[cat].astype(str))        # 범주형 카테고리 정의 합니다.
    cat_arr = le.transform(dataset[cat])    # 범주형 카테고리 숫자로 변환 합니다.

    # 범주형(숫자변환) OneHot Encoder 사용 합니다.
    ohe = preprocessing.OneHotEncoder(categories='auto')     # OneHotEncoder 사용을 위해 선언합니다.
    ohe.fit(cat_arr.reshape(-1,1))

    # 학습과 예측(서비스) 사용위하 le & ohe Object로 저장합니다.
    oh_encoder['LabelEncoder'] = le
    oh_encoder['OneHotEncoder'] = ohe

    # 항목별 OneHot 정보 저장합니다.
    preprocess_object[cat] = oh_encoder

    # le & ohe 삭제 합니다
    del ohe
    del le

# 데이터 전처리 환경 파일(pickle)로 저장 합니다.
ca.log_print('전처리 환경 저장 합니다.')
with open('./output/preprocess_object.pickle', 'wb') as f:
    pickle.dump(preprocess_object, f, pickle.HIGHEST_PROTOCOL)



--------------------------------------------------
OneHot Encoding 처리 합니다.
--------------------------------------------------
--------------------------------------------------
전처리 환경 저장 합니다.
--------------------------------------------------
